In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
Y_MOVIE_URL = 'https://movies.yahoo.com.tw/movie_comingsoon.html'

# 以下網址後面加上 "/id=MOVIE_ID" 即為該影片各項資訊
Y_INTRO_URL = 'https://tw.movies.yahoo.com/movieinfo_main.html'  # 詳細資訊
Y_PHOTO_URL = 'https://tw.movies.yahoo.com/movieinfo_photos.html'  # 劇照
Y_TIME_URL = 'https://tw.movies.yahoo.com/movietime_result.html'  # 時刻表

def get_web_page(url):
    try:
        resp = requests.get(url)
        if resp.status_code == 200:
            return resp.text
        else:
            return None
    except:
        return None

def get_movies(dom):
    soup = BeautifulSoup(dom, 'html5lib')
    movies = []
    rows = soup.find_all('div', 'release_info_text')
    for row in rows:
        movie = dict()
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        movie['movie_id'] = get_movie_id(row.find('div', 'release_movie_name').a['href'])
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        movie['release_date'] = get_date(row.find('div', 'release_movie_time').text)
        movie['intro'] = row.find('div', 'release_text').text.replace(u'詳全文', '').strip()
        trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        movies.append(movie)
    return movies


def get_date(date_str):
    # e.g. "上映日期：2017-03-23" -> match.group(0): "2017-03-23"
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)


def get_movie_id(url):
    # 20180515: URL 格式有變, e.g., 'https://movies.yahoo.com.tw/movieinfo_main/%E6%AD%BB%E4%BE%8D2-deadpool-2-7820.html
    # e.g., "https://tw.rd.yahoo.com/referurl/movie/thisweek/info/*https://tw.movies.yahoo.com/movieinfo_main.html/id=6707"
    #       -> match.group(0): "/id=6707"
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id


def get_trailer_url(url):
    # e.g., 'https://tw.rd.yahoo.com/referurl/movie/thisweek/trailer/*https://tw.movies.yahoo.com/video/美女與野獸-最終版預告-024340912.html'
    return url.split('*')[1]


def get_complete_intro(movie_id):
    page = get_web_page(Y_INTRO_URL + '/id=' + movie_id)
    if page:
        soup = BeautifulSoup(page, 'html5lib')
        infobox = soup.find('div', 'gray_infobox_inner')
        return infobox.text.replace(u'詳全文', '').strip()

if __name__ == '__main__':
    page = get_web_page(Y_MOVIE_URL)
    if page:
        movies = get_movies(page)
        with open('movie.json', 'w', encoding='utf-8') as f:
            json.dump(movies, f, indent=2, sort_keys=True, ensure_ascii=False)
        print('電影介紹被div的gray_infobox_inner包圍')
        for movie in movies:
            intro=get_complete_intro(movie["movie_id"])
            print('intro:',intro,'\n')
        print('\n')
        movies=pd.DataFrame.from_dict(movies)
        print(movies)        

電影介紹被div的gray_infobox_inner包圍
intro: 漫威影業2020年的首部重頭戲，是粉絲請求多時，終於實現的《黑寡婦》個人電影，黑寡婦在漫威電影宇宙中舉足輕重，10多年來參與了7部電影。現在，娜塔莎羅曼諾夫，也就是大家所熟知的黑寡婦，終於踏上個人的旅程，讓粉絲看到她不為人知的一面。
 
故事時間點設定在《美國隊長 3 : 英雄內戰》之後，娜塔莎因為幫助了美國隊長而踏上流亡之路，當她發現一個與過去有關的陰謀時，她必須全球追蹤，回頭面對她神祕的間諜生涯，同時逃過反派「模仿大師」的追殺。 

intro: 詹皇和他年幼的兒子小唐被一個抓狂的人工智慧困在一個數位空間，他必須登上球場，帶領兔寶寶、蘿拉兔及一幫調皮的「樂一通」（Looney Tunes）角色，打敗人工智慧設計出的冠軍球隊，才能讓他和他兒子安全返家。然而，這支數位球隊是集結NBA和WNBA球星的超級強隊，絕對是空前未見。這場硬戰是詹皇人生中玩最大的挑戰，也會重新定他與他兒子的緊密關係，並發揮他重新看見自己的力量。準備放手一搏的「樂一通」角色將打破慣例，讓他們的各種奇招衝到破表，打出一場他們專屬的球賽，連詹皇都會大吃一驚。 

intro: 為了將害人致死的「世界頂尖密室逃脫」主辦方繩之以法，兩位遊戲生存者柔伊（泰勒羅素飾）和班（羅根米勒 飾）決定聯手到紐約找出背後兇手。但兩人卻與其他四位陌生人在紐約地鐵上遇到大災難，並意外發現他們不僅捲入更危險致命的密室遊戲，且所有人之前都是「世界頂尖密室逃脫」遊戲的生存者。在不解迷就等死的情況下，這六人將如何協力找生路，避免成為下一批遊戲犧牲者？ 

intro: 《厲陰宅3：是惡魔逼我的》改編自華倫夫婦的真實檔案，該片揭露了一則集恐怖、謀殺與未知惡魔，令人寒毛直豎的故事，讓即便身為資深超自然現象調查者的艾德和羅琳華倫也感到害怕。在這個他們偵辦過最聳人聽聞的案件中，一開始他們要拯救一名年輕男孩的靈魂，接著他們經歷了前所未見的遭遇，並遇見了全美第一位以惡魔附身做為殺人抗辯理由的謀殺嫌犯。 

intro: 馮迪索飾演的唐老大和莉蒂與他的兒子小布萊恩，正在過著與世隔絕的平靜生活，但是他們都知道這種平靜的生活隨時隨地都會受到致命危機的威脅。這次，這個威脅將逼迫唐老大面對他過去犯下的罪孽，如果他想要拯救他最愛的人們。他的團隊將再度攜手合作，槓上他們遇過最高竿的職